In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [3]:
target = train.TARGET.values.copy()
train.drop('TARGET', axis=1, inplace=True)

In [4]:
train.loc[:, 'n0'] = (train == 0).sum(axis=1)
test.loc[:, 'n0'] = (test == 0).sum(axis=1)

In [5]:
dlist = list()
for c in train.columns:
    if train[c].unique().shape[0] == 1:
        print(c)
        dlist.append(c)
train.drop(dlist, axis=1, inplace=True)
test.drop(dlist, axis=1, inplace=True)

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var33_hace3
imp_trasp_var17_out_hace3
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var33_hace3
num_trasp_var17_out_hace3
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3


In [7]:
for cone in train.columns:
    for ctwo in train.columns:
        if cone not in train.columns:
            continue
        if ctwo not in train.columns:
            continue
        if cone != ctwo and np.all(train[cone] == train[ctwo]):
            print(cone, ctwo)
            train.drop(ctwo, axis=1, inplace=True)
            test.drop(ctwo, axis=1, inplace=True)

ind_var6_0 ind_var29_0
ind_var6 ind_var29
ind_var13_medio_0 ind_var13_medio
ind_var18_0 ind_var18
ind_var26_0 ind_var26
ind_var25_0 ind_var25
ind_var32_0 ind_var32
ind_var34_0 ind_var34
ind_var37_0 ind_var37
ind_var40 ind_var39
num_var6_0 num_var29_0
num_var6 num_var29
num_var13_medio_0 num_var13_medio
num_var18_0 num_var18
num_var26_0 num_var26
num_var25_0 num_var25
num_var32_0 num_var32
num_var34_0 num_var34
num_var37_0 num_var37
num_var40 num_var39
saldo_var6 saldo_var29
saldo_var13_medio saldo_medio_var13_medio_ult1
delta_imp_reemb_var13_1y3 delta_num_reemb_var13_1y3
delta_imp_reemb_var17_1y3 delta_num_reemb_var17_1y3
delta_imp_reemb_var33_1y3 delta_num_reemb_var33_1y3
delta_imp_trasp_var17_in_1y3 delta_num_trasp_var17_in_1y3
delta_imp_trasp_var17_out_1y3 delta_num_trasp_var17_out_1y3
delta_imp_trasp_var33_in_1y3 delta_num_trasp_var33_in_1y3
delta_imp_trasp_var33_out_1y3 delta_num_trasp_var33_out_1y3


In [10]:
for c in train.columns:
    cmin = train[c].min()
    cmax = train[c].max()
    test.loc[test[c] < cmin, c] = cmin
    test.loc[test[c] > cmax, c] = cmax

In [11]:
import xgboost as xgb

In [12]:
dtrain = xgb.DMatrix(train, target)

In [13]:
params = {'objective': 'binary:logistic', 
          'eval_metric': 'auc',
          'eta': 0.0202048,
          'max_depth': 5,
          'subsample': 0.6815,
          'colsample_bytree': 0.701,
          'silent': 1,
          'seed': 0
}

In [14]:
xgb.cv(params, dtrain, num_boost_round=560, nfold=5, stratified=True, verbose_eval=True)

[0]	cv-test-auc:0.7710182+0.05133265367931019	cv-train-auc:0.7820258+0.03944955842287718
[1]	cv-test-auc:0.7813483999999999+0.0392744984365173	cv-train-auc:0.7952296+0.03167585906396225
[2]	cv-test-auc:0.7874858+0.03652494261953057	cv-train-auc:0.8007197999999999+0.030266051538976792
[3]	cv-test-auc:0.7968794+0.03487682282318733	cv-train-auc:0.8096522+0.028841792332655065
[4]	cv-test-auc:0.7981248+0.03568184628294897	cv-train-auc:0.8118896000000001+0.029711415756237532
[5]	cv-test-auc:0.7985434+0.03679613541990519	cv-train-auc:0.8135444000000002+0.03009065662028664
[6]	cv-test-auc:0.8096292+0.019895812075911855	cv-train-auc:0.8235178+0.015109538926122145
[7]	cv-test-auc:0.813953+0.013845096171569218	cv-train-auc:0.8269388+0.011003635879108327
[8]	cv-test-auc:0.8166040000000001+0.012299740696453704	cv-train-auc:0.8291142+0.008570934380801207
[9]	cv-test-auc:0.8177306+0.011964338052729883	cv-train-auc:0.8302956000000001+0.008003724498007186
[10]	cv-test-auc:0.8168840000000002+0.013560792

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.771018,0.051333,0.782026,0.039450
1,0.781348,0.039274,0.795230,0.031676
2,0.787486,0.036525,0.800720,0.030266
3,0.796879,0.034877,0.809652,0.028842
4,0.798125,0.035682,0.811890,0.029711
5,0.798543,0.036796,0.813544,0.030091
6,0.809629,0.019896,0.823518,0.015110
7,0.813953,0.013845,0.826939,0.011004
8,0.816604,0.012300,0.829114,0.008571
9,0.817731,0.011964,0.830296,0.008004


In [15]:
dtest = xgb.DMatrix(test)

In [16]:
gbm = xgb.train(params, dtrain, num_boost_round=560)

In [17]:
preds = gbm.predict(dtest)

In [18]:
preds

array([ 0.04708761,  0.05199018,  0.00153792, ...,  0.00349039,
        0.05911798,  0.00108599], dtype=float32)

In [26]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = preds
sample.to_csv('submission/init_xgb.csv', index_label='ID')